# ServiceNow Matplotlib Demo
This notebook uses **pandas** and **matplotlib** to visualize a synthetic ServiceNow incident dataset.
It generates 10 different charts for analysis.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('servicenow_incidents.csv', parse_dates=['opened_at','resolved_at'])
df['opened_date'] = df['opened_at'].dt.date
df['opened_hour'] = df['opened_at'].dt.hour
df['opened_wday'] = df['opened_at'].dt.day_name()

# Ensure inline display in Jupyter
%matplotlib inline


## 1) Incidents by Priority (Bar)

In [ ]:
counts = df['priority'].value_counts().sort_index()
plt.figure()
counts.plot(kind='bar')
plt.title('Incidents by Priority')
plt.xlabel('Priority')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## 2) Incidents Opened per Day (Line)

In [ ]:
daily = df.groupby('opened_date').size()
plt.figure()
daily.plot()
plt.title('Incidents Opened per Day')
plt.xlabel('Date')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## 3) Mean Time to Resolve by Priority (Bar)

In [ ]:
ttr = df.dropna(subset=['time_to_resolve_minutes']).groupby('priority')['time_to_resolve_minutes'].mean().reindex(['P1','P2','P3','P4'])
plt.figure()
ttr.plot(kind='bar')
plt.title('Mean Time to Resolve (minutes) by Priority')
plt.xlabel('Priority')
plt.ylabel('Minutes')
plt.tight_layout()
plt.show()

## 4) SLA Breach % by Assignment Group (Top 10)

In [ ]:
sla = (df.groupby('assignment_group')['sla_breached'].mean()*100).sort_values(ascending=False).head(10)
plt.figure()
sla.plot(kind='bar')
plt.title('SLA Breach Percentage by Assignment Group (Top 10)')
plt.xlabel('Assignment Group')
plt.ylabel('Breach %')
plt.tight_layout()
plt.show()

## 5) Incident States by Category (Stacked Bar)

In [ ]:
pivot = pd.pivot_table(df, index='category', columns='state', values='incident_id', aggfunc='count', fill_value=0)
pivot = pivot[['New','In Progress','On Hold','Resolved','Closed','Cancelled']]
plt.figure()
pivot.plot(kind='bar', stacked=True)
plt.title('Incident States by Category')
plt.xlabel('Category')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## 6) Heatmap: Opened Hour vs Weekday

In [ ]:
heat = pd.pivot_table(df, index='opened_wday', columns='opened_hour', values='incident_id', aggfunc='count', fill_value=0)
heat = heat.reindex(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.figure()
plt.imshow(heat.values, aspect='auto')
plt.xticks(range(len(heat.columns)), heat.columns, rotation=90)
plt.yticks(range(len(heat.index)), heat.index)
plt.title('Incidents Opened: Hour vs Weekday')
plt.colorbar()
plt.tight_layout()
plt.show()

## 7) TTR Distribution by Category (Boxplot)

In [ ]:
resolved = df.dropna(subset=['time_to_resolve_minutes'])
groups = [resolved.loc[resolved['category']==cat,'time_to_resolve_minutes'] for cat in resolved['category'].unique()]
labels = list(resolved['category'].unique())
plt.figure()
plt.boxplot(groups, labels=labels, showfliers=False)
plt.title('Time to Resolve by Category')
plt.xlabel('Category')
plt.ylabel('Minutes')
plt.tight_layout()
plt.show()

## 8) Pareto: Top 10 Configuration Items

In [ ]:
top = df['configuration_item'].value_counts().head(10)
cum = top.cumsum()/top.sum()*100
plt.figure()
ax = top.plot(kind='bar')
ax2 = ax.twinx()
ax2.plot(range(len(top)), cum.values, marker='o')
ax2.set_ylim(0,100)
ax.set_title('Pareto: Top 10 Configuration Items')
ax.set_xlabel('Configuration Item')
ax.set_ylabel('Count')
ax2.set_ylabel('Cumulative %')
plt.tight_layout()
plt.show()

## 9) Workflow Funnel by State (Bar)

In [ ]:
state_order = ['New','In Progress','On Hold','Resolved','Closed','Cancelled']
counts = df['state'].value_counts().reindex(state_order).fillna(0)
plt.figure()
counts.plot(kind='barh')
plt.title('Workflow Funnel by State')
plt.xlabel('Count')
plt.ylabel('State')
plt.tight_layout()
plt.show()

## 10) First Response Time vs Customer Satisfaction (Scatter)

In [ ]:
scatter_df = df.dropna(subset=['customer_satisfaction'])
plt.figure()
plt.scatter(scatter_df['time_to_first_response_minutes'], scatter_df['customer_satisfaction'])
plt.title('First Response Time vs CSAT')
plt.xlabel('Time to First Response (minutes)')
plt.ylabel('CSAT (1-5)')
plt.tight_layout()
plt.show()